In [ ]:
import io
import json
import os
from functools import reduce
import h5py
import numpy as np
import trimesh
from PIL import Image
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

In [ ]:
config = 'pilot_dominoes_1mid_J025R45_boxroom'
trial_index = str(3)

gt_hdf5_file_path = f'/home/haoliangwang/data/physion_hdf5/dominoes_all_movies/{config}_{trial_index.zfill(4)}.hdf5'
with open(f'/home/haoliangwang/data/b3d_tracking_results/test/dominoes_verbose/{config}_{trial_index.zfill(4)}.json') as f:
    tracking_per_frame = json.load(f)
with open(f'/home/haoliangwang/data/b3d_tracking_results/test/dominoes/{config}_{trial_index.zfill(4)}.json') as f:
    tracking_final_frame = json.load(f)


In [ ]:
NUM_SAMPLE_FROM_POSTERIOR = 20
SMOOTHING_WINDOW_SIZE = 3
FPS = 100
STATIC_POSITION_THRESHHOLD = 0.007
STATIC_ROTATION_THRESHHOLD = 0.001

def compute_angular_velocity(q1, q2, dt):
    # Convert quaternions to scipy Rotation objects
    rot1 = Rotation.from_quat(q1)
    rot2 = Rotation.from_quat(q2)

    # Compute the relative rotation
    relative_rotation = rot2 * rot1.inv()

    # Convert the relative rotation to angle-axis representation
    angle = relative_rotation.magnitude()  # Rotation angle in radians
    axis = (
        relative_rotation.as_rotvec() / angle if angle != 0 else np.zeros(3)
    )  # Rotation axis

    # Compute angular velocity
    angular_velocity = (axis * angle) / dt
    return {
        "x": angular_velocity[0].astype(float).item(),
        "y": angular_velocity[1].astype(float).item(),
        "z": angular_velocity[2].astype(float).item(),
    }

def sample_from_posterior(log_probs_categories, option="rank"):
    log_probs = [item[0] for item in log_probs_categories]
    categories = [item[1] for item in log_probs_categories]
    num_categories = len(log_probs)

    if option == "uniform":

        def draw_single_sample():
            index = np.random.choice(num_categories)
            return categories[index]
    elif option == "veridical":

        def draw_single_sample():
            # see this: https://stackoverflow.com/questions/58339083/how-to-sample-from-a-log-probability-distribution
            gumbels = np.random.gumbel(size=num_categories)
            index = np.argmax(log_probs + gumbels)
            return categories[index]
    elif option == "rank":

        def draw_single_sample():
            weights = np.array([1 / (n + 1) for n in range(num_categories)])
            weights_norm = weights / weights.sum()
            index = np.random.choice(num_categories, p=weights_norm)
            return categories[index]
    elif option == "mix":

        def draw_single_sample():
            t = 0.5
            t * np.array(log_probs) + (1 - t) * (1 / num_categories)
            return
    else:
        raise NotImplementedError

    samples = []
    np.random.seed(42)
    for _ in range(NUM_SAMPLE_FROM_POSTERIOR):
        sample = draw_single_sample()
        samples.append(sample)
    return samples
